# Install lib

In [1]:
!pip install kafka-python
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
from kafka import KafkaConsumer
import json
import pandas as pd

In [3]:
data = []

In [4]:
# use this function to print the json with the indent=4
def jprint(data):
  print(json.dumps(data,indent=4))

# Kafka Consumer

In [6]:
consumer = KafkaConsumer(
    '2021',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    value_deserializer=lambda x: json.loads(x.decode('utf-8')),
)

print("Starting the consumer...")

for message in consumer:
    data.append(message.value)
    print(f"Received: {message.value}")

print("get all data from 2021, done")

Starting the consumer...
Received: {'id': '202100031', 'publicDate': '15/12/2021', 'source': 1, 'coAuthorship': 2, 'citationCount': '0', 'refCount': '32', 'Class': ['MULT']}
Received: {'id': '202100203', 'publicDate': '01/12/2021', 'source': 1, 'coAuthorship': 8, 'citationCount': '5', 'refCount': '58', 'Class': ['MULT']}
Received: {'id': '202100204', 'publicDate': '01/12/2021', 'source': 1, 'coAuthorship': 2, 'citationCount': '11', 'refCount': '36', 'Class': ['BUSI', 'ARTS', 'SOCI', 'PSYC', 'ECON']}
Received: {'id': '202100036', 'publicDate': '15/12/2021', 'source': 1, 'coAuthorship': 5, 'citationCount': '19', 'refCount': '38', 'Class': ['PHAR', 'ENVI']}
Received: {'id': '202100009', 'publicDate': '28/12/2021', 'source': 1, 'coAuthorship': 3, 'citationCount': '3', 'refCount': '60', 'Class': ['PHYS', 'CHEM']}
Received: {'id': '202100232', 'publicDate': '01/12/2021', 'source': 1, 'coAuthorship': 6, 'citationCount': '7', 'refCount': '45', 'Class': ['MEDI', 'BIOC']}
Received: {'id': '20210

KeyboardInterrupt: 

In [7]:
jprint(data)

[
    {
        "id": "202100031",
        "publicDate": "15/12/2021",
        "source": 1,
        "coAuthorship": 2,
        "citationCount": "0",
        "refCount": "32",
        "Class": [
            "MULT"
        ]
    },
    {
        "id": "202100203",
        "publicDate": "01/12/2021",
        "source": 1,
        "coAuthorship": 8,
        "citationCount": "5",
        "refCount": "58",
        "Class": [
            "MULT"
        ]
    },
    {
        "id": "202100204",
        "publicDate": "01/12/2021",
        "source": 1,
        "coAuthorship": 2,
        "citationCount": "11",
        "refCount": "36",
        "Class": [
            "BUSI",
            "ARTS",
            "SOCI",
            "PSYC",
            "ECON"
        ]
    },
    {
        "id": "202100036",
        "publicDate": "15/12/2021",
        "source": 1,
        "coAuthorship": 5,
        "citationCount": "19",
        "refCount": "38",
        "Class": [
            "PHAR",
            "ENVI"
 

# Spark for processing

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame(data)
df = df.select("id", "publicDate", "source", "citationCount", "coAuthorship", "refCount", "Class") # reordering
df.show(10)

24/05/07 22:43:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/07 22:43:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+---------+----------+------+-------------+------------+--------+--------------------+
|       id|publicDate|source|citationCount|coAuthorship|refCount|               Class|
+---------+----------+------+-------------+------------+--------+--------------------+
|202100031|15/12/2021|     1|            0|           2|      32|              [MULT]|
|202100203|01/12/2021|     1|            5|           8|      58|              [MULT]|
|202100204|01/12/2021|     1|           11|           2|      36|[BUSI, ARTS, SOCI...|
|202100036|15/12/2021|     1|           19|           5|      38|        [PHAR, ENVI]|
|202100009|28/12/2021|     1|            3|           3|      60|        [PHYS, CHEM]|
|202100232|01/12/2021|     1|            7|           6|      45|        [MEDI, BIOC]|
|202100000|31/12/2021|     1|            0|           2|      45|              [MEDI]|
|202100007|30/12/2021|     1|            4|           1|      24|  [BUSI, SOCI, ECON]|
|202100235|01/12/2021|     1|            8|

In [22]:
df.count()

301

In [23]:
# cast type accordingly
df = df.withColumn('citationCount', df.citationCount.cast('int'))
df = df.withColumn('coAuthorship', df.coAuthorship.cast('int'))
df = df.withColumn('refCount', df.refCount.cast('int'))

In [24]:
from pyspark.sql.functions import avg, min, max, countDistinct, explode, split, col, round, sum

max_values = df.agg(
    max("citationCount").alias("maxCitation"),
    max("refCount").alias("maxRef"),
    max("coAuthorship").alias("maxCoAuthor")
).collect()[0]

# max value for each feature for normalization
max_citation = max_values["maxCitation"]
max_ref = max_values["maxRef"]
max_coauthor = max_values["maxCoAuthor"]

In [25]:
# find all Class
genre_counts = df.withColumn("Genre", explode(col("Class")))\
                 .groupBy("Genre")\
                 .count()  

print('number of all the class:', genre_counts.count())
genre_counts.show()

number of all the class: 27
+-----+-----+
|Genre|count|
+-----+-----+
| COMP|   15|
| IMMU|   16|
| MATE|   24|
| ARTS|    7|
| PSYC|    1|
| PHYS|   21|
| BIOC|   33|
| VETE|   11|
| ENGI|   33|
| PHAR|    9|
| MEDI|   97|
| ECON|    4|
| MULT|   34|
| ENVI|   35|
| AGRI|   29|
| CENG|   25|
| SOCI|   32|
| CHEM|   29|
| BUSI|    6|
| HEAL|    1|
+-----+-----+
only showing top 20 rows



In [26]:
# explode the class
exploded_df = df.withColumn("Class", explode(col("Class")))
# exploded_df.show(20)

In [27]:
# drop na for invalid rows
cleaned_df = exploded_df.dropna()
print(cleaned_df.count())
cleaned_df.show(5)

488
+---------+----------+------+-------------+------------+--------+-----+
|       id|publicDate|source|citationCount|coAuthorship|refCount|Class|
+---------+----------+------+-------------+------------+--------+-----+
|202100031|15/12/2021|     1|            0|           2|      32| MULT|
|202100203|01/12/2021|     1|            5|           8|      58| MULT|
|202100204|01/12/2021|     1|           11|           2|      36| BUSI|
|202100204|01/12/2021|     1|           11|           2|      36| ARTS|
|202100204|01/12/2021|     1|           11|           2|      36| SOCI|
+---------+----------+------+-------------+------------+--------+-----+
only showing top 5 rows



In [28]:
# compute the score for each paper
cleaned_df = cleaned_df.withColumn(
    "Score",
    round(
        col("source") * (
            0.4 * (col("citationCount") / max_citation * 10) +
            0.2 * (col("refCount") / max_ref * 10) +
            0.1 * (col("coAuthorship") / max_coauthor * 10)
        ), 4
    )
)

cleaned_df.show(10)

+---------+----------+------+-------------+------------+--------+-----+------+
|       id|publicDate|source|citationCount|coAuthorship|refCount|Class| Score|
+---------+----------+------+-------------+------------+--------+-----+------+
|202100031|15/12/2021|     1|            0|           2|      32| MULT|0.1821|
|202100203|01/12/2021|     1|            5|           8|      58| MULT|0.4146|
|202100204|01/12/2021|     1|           11|           2|      36| BUSI|0.3866|
|202100204|01/12/2021|     1|           11|           2|      36| ARTS|0.3866|
|202100204|01/12/2021|     1|           11|           2|      36| SOCI|0.3866|
|202100204|01/12/2021|     1|           11|           2|      36| PSYC|0.3866|
|202100204|01/12/2021|     1|           11|           2|      36| ECON|0.3866|
|202100036|15/12/2021|     1|           19|           5|      38| PHAR|0.5315|
|202100036|15/12/2021|     1|           19|           5|      38| ENVI|0.5315|
|202100009|28/12/2021|     1|            3|         

In [29]:
from pyspark.sql.functions import to_date, year, quarter, col, sum, round, count
from pyspark.sql.window import Window

cleaned_df = cleaned_df.withColumn("publicDate", to_date(col("publicDate"), "dd/MM/yyyy"))
cleaned_df = cleaned_df.withColumn("Year", year(col("publicDate")))

grouped_df = cleaned_df.groupBy("Class", "Year").agg(
    round(sum("Score"), 4).alias("Total Score"),
    count("id").alias("Paper Count")  
)

# Display the result
grouped_df.show()

+-----+----+-----------+-----------+
|Class|Year|Total Score|Paper Count|
+-----+----+-----------+-----------+
| PSYC|2021|     0.3866|          1|
| VETE|2021|     2.5835|         11|
| ECON|2021|      3.697|          4|
| PHAR|2021|     6.4303|          9|
| ENVI|2021|    18.4076|         33|
| CHEM|2021|    16.7929|         29|
| MEDI|2021|    46.1412|         95|
| MATE|2021|    10.9868|         24|
| AGRI|2021|    12.5653|         24|
| IMMU|2021|    12.7477|         15|
| MULT|2021|    16.4315|         33|
| ENGI|2021|    15.3488|         32|
| CENG|2021|    14.8443|         24|
| BIOC|2021|    19.8479|         32|
| ARTS|2021|     1.3135|          6|
| COMP|2021|     7.0748|         15|
| PHYS|2021|    15.1241|         20|
| BUSI|2021|     2.1637|          6|
| SOCI|2021|    11.2682|         31|
| EART|2021|     7.3572|         13|
+-----+----+-----------+-----------+
only showing top 20 rows



In [30]:
# Save the DataFrame to a CSV file
cumulative_df.write.csv(path="output_2021.csv", mode="overwrite", header=True)

spark.stop()

Py4JJavaError: An error occurred while calling o324.csv.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:75)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:484)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con